# Predicting the Residuary Resistance of Sailing Yachts

1. **Frame the problem and look at the big picture**
2. **Get the Data**
3. **Data Insights**
4. **Prepare the Data for ML Algorithms**
5. **Select and Train a Model**
6. **Model Fine Tuning**
7. **Test Set Evaluation**

In [ ]:
import sys
print("Python version:", sys.version_info)

In [ ]:
import sklearn
print("scikit-learn version:", sklearn.__version__)

# 2. Get the Data

In [ ]:
import os
import urllib.request

In [ ]:
DOWNLOAD_ROOT = "http://archive.ics.uci.edu/ml/machine-learning-databases/00243/"
DATA_URL = DOWNLOAD_ROOT + "yacht_hydrodynamics.data"
DATA_PATH = os.path.join("datasets", "yacht")

In [ ]:
def fetch_data(data_url=DATA_URL, data_path=DATA_PATH):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    full_data_path = os.path.join(data_path, "yacht_hydrodynamics.data")
    urllib.request.urlretrieve(data_url, full_data_path)

fetch_data()

In [ ]:
import pandas as pd

columns = ["Centre_of_Buoyancy", "Prismatic_Coefficient", "Length/Displacement_Ratio", "Beam/Draft_Ratio", 
           "Length/Beam_Ratio", "Froude_Number", "Residuary_Resistance"]

def load_data(data_path=DATA_PATH):
    full_data_path = os.path.join(data_path, "yacht_hydrodynamics.data")
    return pd.read_table(full_data_path, sep='\s+', names=columns)

yacht = load_data()

# 3. Data Insights

## The Data Structure

In [ ]:
# look at the top ten rows
yacht.head(10)

In [ ]:
# get a quick description of the data:
# the total number of rows, each attribute's type, and the number of non-null values
yacht.info()

There are 308 instances in the dataset, which means that it is fairly small by ML standards, but the aim of this project is to apply machine learning in the domain of naval engineering.

In [ ]:
# the number of null values for each attribute
yacht.isnull().sum()

In [ ]:
# 22 sailing yacht hull forms were tested at the same speeds within the speed range 0.125 - 0.450
yacht["Froude_Number"].value_counts()

In [ ]:
# get a summary of the numerical attributes
yacht.describe().round(3)

The std row shows the standard deviation, which measures how dispersed the values are.

The 25%, 50% and 75% rows show the corresponding percentiles, which indicate the values below which a given percentage of observations in a group of observations fall. 

For example, 50% of the hull forms have the residuary resistance lower than 3.065, while the max value is 62.42.

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rc("font", size=14)
mpl.rc("axes", labelsize=14, titlesize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

# for each attribute, show the number of instances (on the vertical axis) that have a given value range (on the horizontal axis)
yacht.hist(bins=3, figsize=(15, 10))

In [ ]:
yacht.plot(kind="scatter", x="Froude_Number", y="Residuary_Resistance")

**The residuary resistance grows eponentially with increase of the velocity.**

## Create a Test set

In [ ]:
yacht["Length/Displacement_Ratio"].value_counts() / len(yacht)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# stratified sampling based on the length/displacement ratio
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(yacht, yacht["Length/Displacement_Ratio"]):
    strat_train_set = yacht.loc[train_index]
    strat_test_set = yacht.loc[test_index]

In [ ]:
yacht_features = strat_train_set.drop("Residuary_Resistance", axis=1)
yacht_labels = strat_train_set["Residuary_Resistance"].copy()

### Data Correlations

In [ ]:
yacht.corr()

In [ ]:
corr_matrix = yacht.corr()

corr_matrix["Residuary_Resistance"].sort_values(ascending=False)

# 4. Prepare the Data for ML Algorithms

### Training and test set

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(yacht, test_size=0.2, random_state=42)

train_set.head()

In [ ]:
train_set.info()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# stratified sampling based on the length/displacement ratio
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(yacht, yacht["Length/Displacement_Ratio"]):
    strat_train_set = yacht.loc[train_index]
    strat_test_set = yacht.loc[test_index]

In [ ]:
yacht_features = strat_train_set.drop("Residuary_Resistance", axis=1)
yacht_labels = strat_train_set["Residuary_Resistance"].copy()

In [ ]:
sample_incomplete_rows = yacht_features[yacht_features.isnull().any(axis=1)]
sample_incomplete_rows

### Custom Transformers

In [ ]:
column_names = "Centre_of_Buoyancy", "Prismatic_Coefficient", "Length/Displacement_Ratio"
# get the column indices
L_bc_ix, Cp_ix, L_D_ratio_ix = [yacht_features.columns.get_loc(c) for c in column_names]
L_bc_ix, Cp_ix, L_D_ratio_ix

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# add two more attributes:
# Lbc_Cp_ratio - center of bouyancy/prismatic coefficient ratio
# LD_Cp_ratio - (length/displacement ratio)/prismatic coefficient ratio
class CombinedAtributeAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_LD_Cp_ratio=True):
        self.add_LD_Cp_ratio = add_LD_Cp_ratio
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        Lbc_Cp_ratio = X[:, L_bc_ix] / X[:, Cp_ix]
        if self.add_LD_Cp_ratio:
            LD_Cp_ratio = X[:, L_D_ratio_ix] / X[:, Cp_ix]
            return np.c_[X, Lbc_Cp_ratio, LD_Cp_ratio]
        else:
            return np.c_[X, Lbc_Cp_ratio]

In [ ]:
attribute_adder = CombinedAtributeAdder(add_LD_Cp_ratio=True)
yacht_extra_attr = attribute_adder.transform(yacht_features.values)

yacht_extra_attr = pd.DataFrame(yacht_extra_attr,
                                    columns=list(yacht_features.columns)
                                    +["Centre_of_Buoyancy/Prismatic_Coefficient_Ratio", "Length_Displacement/Prismatic_Coefficient_Ratio"],
                                    index=yacht_features.index)

yacht_extra_attr.head()

In [ ]:
yacht_extra_attr.describe().round(2)

### Transformation Pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

yacht_pipeline = Pipeline([
            ("attributes_adder", CombinedAtributeAdder()),
            ("features_scaler", StandardScaler()),
        ])

In [ ]:
yacht_features_tr = yacht_pipeline.fit_transform(yacht_features.values)

yacht_features_tr

# 5. Select and Train a Model

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(yacht_features_tr, yacht_labels)
yacht_predictions = lin_reg.predict(yacht_features_tr)

lin_mse = mean_squared_error(yacht_labels, yacht_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_mse

In [ ]:
lin_reg.intercept_, lin_reg.coef_

In [ ]:
def plot_learning_curves(model, features, labels):
    features_train, features_val, labels_train, labels_val = train_test_split(features, labels, test_size=0.1, random_state=42)
    train_errors, val_errors = [], []
    for m in range(1, len(features_train) + 1):
        model.fit(features_train[:m], labels_train[:m])
        train_predict = model.predict(features_train[:m])
        val_predict = model.predict(features_val)
        train_errors.append(mean_squared_error(train_predict, labels_train[:m]))
        val_errors.append(mean_squared_error(val_predict, labels_val))
    
    plt.plot(np.sqrt(train_errors), 'r-.', linewidth=3, label='train')
    plt.plot(np.sqrt(val_errors), 'b-', linewidth=3, label='val')
    plt.legend(loc='upper right', fontsize=14)
    plt.xlabel("Training set size", fontsize=14)
    plt.ylabel("RMSE", fontsize=14)
    print("Training error:", np.sqrt(train_errors[-1]))
    print("Vallidation error:", np.sqrt(val_errors[-1]))

In [ ]:
plot_learning_curves(lin_reg, yacht_features_tr, yacht_labels)
plt.axis([0, 220, 0, 20])
plt.show()

In [ ]:
lin_reg.intercept_, lin_reg.coef_

### Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

polynomial_regression = Pipeline([
        ("attributes_adder", CombinedAtributeAdder()),
        ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
        ("features_scaler", StandardScaler()),
        ("lin_reg", LinearRegression()),
    ])

In [ ]:
plot_learning_curves(polynomial_regression, yacht_features.values, yacht_labels)
plt.show()

### Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)

In [ ]:
plot_learning_curves(elastic_net, yacht_features_tr, yacht_labels)
plt.show()

### Stohastic Gradient Descent - Early Stopping

In [ ]:
from sklearn.linear_model import SGDRegressor
from copy import deepcopy

features_train, features_val, labels_train, labels_val = train_test_split(yacht_features.values, yacht_labels.ravel(), test_size=0.2, random_state=42)

polynomial_scaler = Pipeline([
        ("attributes_adder", CombinedAtributeAdder()),
        ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
        ("features_scaler", StandardScaler()),
])

features_train_poly_scaled = polynomial_scaler.fit_transform(features_train)
features_val_poly_scaled = polynomial_scaler.transform(features_val)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

n_epochs = 500
train_errors, val_errors = [], []

minimum_val_error = float("inf")
best_epoch = None
best_model = None

for epoch in range(n_epochs):
    sgd_reg.fit(features_train_poly_scaled, labels_train)
    y_train_predict = sgd_reg.predict(features_train_poly_scaled)
    train_errors.append(mean_squared_error(labels_train, y_train_predict))

    y_val_predict = sgd_reg.predict(features_val_poly_scaled)
    val_error = mean_squared_error(labels_val, y_val_predict)
    val_errors.append(val_error)

    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = deepcopy(sgd_reg)

best_train_epoch = np.argmin(train_errors)
best_train_rmse = np.sqrt(train_errors[best_train_epoch])

best_val_epoch = np.argmin(val_errors)
best_val_rmse = np.sqrt(val_errors[best_val_epoch])

In [ ]:
plt.annotate('Best model',
            xy=(best_val_epoch, best_val_rmse),
            xytext=(best_val_epoch, best_val_rmse + 1),
            ha="center",
            arrowprops=dict(facecolor='black', shrink=0.05),
            fontsize=16,
            )

plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], 'k:', linewidth=2)
plt.plot(np.sqrt(val_errors), 'b-', linewidth=3, label="Validation set")
plt.plot(np.sqrt(train_errors), 'r--', linewidth=2, label="Training set")
plt.legend(loc="upper right", fontsize=14)
plt.axis([0, 500, 0, 5])
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("RMSE", fontsize=14)
plt.show()

print("Training error:", best_train_rmse)
print("Vallidation error:", best_val_rmse)

### Support Vector Machines

In [ ]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)
svm_reg.fit(yacht_features_tr, yacht_labels)

In [ ]:
svm_yacht_predictions = svm_reg.predict(yacht_features_tr)
svm_mse = mean_squared_error(yacht_labels, svm_yacht_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

In [ ]:
plot_learning_curves(svm_reg, yacht_features_tr, yacht_labels)

### Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(yacht_features_tr, yacht_labels)

In [ ]:
tree_yacht_predictions = tree_reg.predict(yacht_features_tr)
tree_mse = mean_squared_error(yacht_labels, tree_yacht_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
plot_learning_curves(tree_reg, yacht_features_tr, yacht_labels)

In [ ]:
from sklearn.model_selection import cross_val_score

tree_mse_scores = cross_val_score(tree_reg, yacht_features_tr, yacht_labels,
                        scoring="neg_mean_squared_error", cv=10)

tree_rmse_scores = np.sqrt(-tree_mse_scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

### Random Forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(yacht_features_tr, yacht_labels)

In [ ]:
forest_yacht_predictions = forest_reg.predict(yacht_features_tr)
forest_mse = mean_squared_error(yacht_labels, forest_yacht_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
plot_learning_curves(forest_reg, yacht_features_tr, yacht_labels)

In [ ]:
forest_mse_scores = cross_val_score(forest_reg, yacht_features_tr, yacht_labels,
                        scoring="neg_mean_squared_error", cv=10)

forest_rmse_scores = np.sqrt(-forest_mse_scores)
display_scores(forest_rmse_scores)

# 6. Model Fine Tuning

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [100, 250, 500], 'max_features': [4, 6, 8]},
]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
                           
grid_search.fit(yacht_features_tr, yacht_labels)                          

In [ ]:
print('The best hyperparameter combination found:', grid_search.best_params_)
grid_search.best_estimator_

In [ ]:
grid_search_score = grid_search.cv_results_
for mean_score, params in zip(grid_search_score["mean_test_score"], grid_search_score["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

### Randomized Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distributions = {
    'n_estimators': randint(low=50, high=250),
    'max_features': randint(low=5, high=8),
}

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distributions,
                                n_iter=10, cv= 10, scoring='neg_mean_squared_error', random_state=42)

rnd_search.fit(yacht_features_tr, yacht_labels)

In [ ]:
rnd_search_score = rnd_search.cv_results_
for mean_score, params in zip(rnd_search_score["mean_test_score"], rnd_search_score["params"]):
    print(np.sqrt(-mean_score), params)

# 7. Test Set Evaluation

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("R_rs", axis=1)
y_test = strat_test_set["R_rs"].copy()

X_test_tr = yacht_pipeline.transform(X_test.values)
final_predictions = final_model.predict(X_test_tr)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print('Test set RMSE:', final_rmse)

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

In [ ]:
error_percentage = (abs(y_test - final_predictions) * 100 )/y_test

In [ ]:
error_table = np.block([[final_predictions.round(2)], [y_test], [error_percentage.round(2)]]).T

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
pd.DataFrame(error_table, columns=["Predicted values", "Correct values", "Error (%)"])

In [ ]:
error_percentage.hist(bins=50)

In [ ]:
# the generalization error that the model makes
print("The mean error:", error_percentage.mean().round(2), "%")